In [1]:
import numpy as np
import json
import yaml
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from matplotlib.ticker import MaxNLocator

import xobjects as xo
import xtrack as xt
import xpart as xp
import xcoll as xc


import pymadx
#from xcoll_plotting import plot_functions as xcp

print('\nxcoll version: ', xc.__version__)
print('xtrack version: ', xt.__version__)
print('xpart version: ', xp.__version__)
print('xobjects version: ', xo.__version__)

context = xo.ContextCpu() 
seed = 1927034333


xcoll version:  0.5.12
xtrack version:  0.83.0
xpart version:  0.23.0
xobjects version:  0.5.0


In [2]:
# -------------- CONSTANT PARAMETERS -----------------
beam =  2 
plane = 'V' 
emittance_n = 3.5e-6
layout_file = "../input_files/2023_Run3_flat_top/layout_flat_top_b2.tfs"  
layout_data_object = pymadx.Data.Tfs(layout_file)


TCCP_bending_angle = 6921.3e-6
TCCS_bending_angle = 50e-6
TCCP_length = 0.070
TARGET_length =  0.005
TCCS_length = 0.004

TCCS_name = 'tccs.5r3.b2'
TCCP_name = 'tccp.4l3.b2'
TARGET_name = 'target.4l3.b2'
PIXEL_name = 'pixel.detector'
TFT_name = 'tft.detector'
TCP_name = 'tcp.d6r7.b2'
TCLA_name = 'tcla.a5l3.b2'

TCCS_loc_abs  = 6773.9428  #6773.7 #6775
TCCP_loc_abs  = 6653.2543  #6653.3 #6655
PIX1_loc_abs = 6652.7039
PIX2_loc_abs = 6652.6929
PIX3_loc_abs = 6652.6819
TFT_loc_abs = 6652.114

pymadx.Tfs.Load> normal file


In [3]:
# -------------- SETTABLE PARAMETERS -----------------

coll_file = '../input_files/colldbs/TWOCRYST_colldb_450GeV_NOCRY.yaml'
#coll_file = './input_files/colldbs/TWOCRYST_colldb_450GeV.yaml'
#line_file = './MadX/2024/track_injection_b2.json'
line_file = '../MadX/2025_new/injection/no_twocryst/track_injection_b2.json'
#line_file = './MadX/2025_new/flat_top/track_flat_top_b2.json'

part_energy = 450e9


In [5]:
# ---------------- BUILD LINE ----------------

line = xt.Line.from_json(line_file)
if part_energy is not None:
    line.particle_ref = xt.Particles(p0c=part_energy, #eV
                                q0=1, mass0=xt.PROTON_MASS_EV)
print(f'\nParticle energy: {float(line.particle_ref.p0c)/1e9:} GeV\n')
energy = line.particle_ref.p0c[0]
beta_rel = line.particle_ref.beta0[0]
gamma = line.particle_ref.gamma0[0]
emittance = emittance_n/(beta_rel*gamma)


dp = 1.92e-10 
pot_crit = 21.34
eta = 0.9
bending_radius = TCCS_length/TCCS_bending_angle
Rcrit = energy/(2*np.sqrt(eta)*pot_crit) * (dp/2)
TCCS_critical_angle = np.sqrt(2*eta*pot_crit/energy)*(1 - Rcrit/bending_radius)


end_s = line.get_length()


TCCS_loc = end_s - TCCS_loc_abs
TCCP_loc = end_s - TCCP_loc_abs
TARGET_loc = end_s - (TCCP_loc_abs + 0.07/2 + 0.005/2)
PIX1_loc = end_s - PIX1_loc_abs
PIX2_loc = end_s - PIX2_loc_abs
PIX3_loc = end_s - PIX3_loc_abs
TFT_loc = end_s - TFT_loc_abs
TCP_loc = line.get_s_position()[line.element_names.index(TCP_name)]
TCLA_loc = line.get_s_position()[line.element_names.index(TCLA_name)]


line.insert_element(at_s=TCCS_loc, element=xt.Marker(), name=TCCS_name)
line.insert_element(at_s=TCCS_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCS_name+'_aper')
line.insert_element(at_s=TCCS_loc - TCCS_length/2, element=xt.Marker(), name=TCCS_name+'_exit')
line.insert_element(at_s=TCCS_loc + TCCS_length/2, element=xt.Marker(), name=TCCS_name+'_entry')
line.insert_element(at_s=TCCP_loc, element=xt.Marker(), name=TCCP_name)
line.insert_element(at_s=TCCP_loc - TCCP_length/2, element=xt.Marker(), name=TCCP_name+'_exit')
line.insert_element(at_s=TCCP_loc + TCCP_length/2, element=xt.Marker(), name=TCCP_name+'_entry')
line.insert_element(at_s=TCCP_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name=TCCP_name+'_aper')
line.insert_element(at_s=TARGET_loc, element=xt.Marker(), name=TARGET_name)
line.insert_element(at_s=TARGET_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= TARGET_name + '_aper')
line.insert_element(at_s=PIX1_loc, element=xt.Marker(), name=PIXEL_name+'_1')
line.insert_element(at_s=PIX1_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= PIXEL_name+'_1' + '_aper')
line.insert_element(at_s=PIX2_loc, element=xt.Marker(), name=PIXEL_name+'_2')
line.insert_element(at_s=PIX2_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= PIXEL_name+'_2' + '_aper')
line.insert_element(at_s=PIX3_loc, element=xt.Marker(), name=PIXEL_name+'_3')
line.insert_element(at_s=PIX3_loc, element=xt.LimitEllipse(a_squ=0.0016, b_squ=0.0016, a_b_squ=2.56e-06), name= PIXEL_name+'_3' + '_aper')
line.insert_element(at_s=TFT_loc, element=xt.Marker(), name=TFT_name)


colldb = xc.CollimatorDatabase.from_yaml(coll_file, beam=beam, ignore_crystals=False)
colldb.install_everest_collimators(line = line,verbose=True)

# Aperture model check
print('\nAperture model check after introducing collimators:')
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)



# Build the tracker
line.build_tracker()
tw = line.twiss()

Loading line from dict:   0%|          | 0/102293 [00:00<?, ?it/s]

Done loading line from dict.           


/tmp/ipykernel_509/2501751179.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f'\nParticle energy: {float(line.particle_ref.p0c)/1e9:} GeV\n')



Particle energy: 450.0 GeV



Slicing line:   0%|          | 0/102293 [00:00<?, ?it/s]

...

Installing tcsg.d5l7.b2         as EverestCollimator
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Installing tcla.a5l3.b2         as EverestCollimator
Installing tcdqa.a4l6.b2        as EverestCollimator
Installing tcla.d6l7.b2         as EverestCollimator
Installing tcl.4l5.b2           as EverestCollimator
Installing tcspm.b4r7.b2        as EverestCollimator
Installing tcla.6l3.b2          as EverestCollimator
Installing tctpv.4r1.b2         as EverestCollimator
Installing tcsg.a5r7.b2         as EverestCollimator
Installing tcsg.b5l3.b2         as EverestCollimator
Installing tctph.4r1.b2         as EverestCollimator
Installing tctpv.4r8.b2         as EverestCollimator
Installing tcsg.a4l7.b2         as EverestCollimator
Installing tcspm.6l7.b2         as EverestCollimator
Installing tcsg.4l3.b2          as EverestCollimator
Installing tcsp.a4l6.b2         as EverestCollimator
Installing tcp.6r3.b2           as EverestCollimator
Installing tctph.4r2.b2         

Slicing line:   0%|          | 0/102321 [00:00<?, ?it/s]


Aperture model check after introducing collimators:


Checking aperture:   0%|          | 0/102273 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


In [ ]:
TCCS_alfy = 1.6452809259189083
TCCS_bety = 237.10008673014357

TCCP_alfy = -3.061162194699935
TCCP_bety = 283.9205729846146

PIX_bety  = 287.30138012925875
PIX_alfy  = -3.081266638309871

TFT_bety = 290.94938468345964
TFT_alfy = -3.1028138971861075

TCLA_bety = 177.85452549506485
TCLA_alfy = 1.8688529341101034

In [ ]:

def CRY_tw(gap, energy, element = None, tw = None, beta_y = None, alpha_y = None):
    
    if tw is None and beta_y is None and alfy is None:
        print("Pass something")
        return
    
    mp = 0.9382e9
    emittance_n = 3.5e-6
    gamma = energy/mp
    beta = np.sqrt(1-1/(gamma*gamma))
    emittance = emittance_n/(beta*gamma)

    beta_y = tw['bety', element] if tw is not None else beta_y 
    alpha_y = tw['alfy', element] if tw is not  None else alpha_y
    #print('beta_y = ', beta_y, 'alpha_y = ', alpha_y)
    return gap*np.sqrt(emittance*beta_y), -gap*alpha_y*np.sqrt(emittance/beta_y)

In [ ]:
TCCS_gap = 5

Channeling at 4907 urad. TCLA at 9sigma    
Channeling angle:  4910.84964321279

In [ ]:
TCCS_pos_measured = 50.5 
TCCS_ch_angle_mesured = 4910.84964321279 *1e-6 # 4907

In [ ]:
def calculate_expected_chann_angle(new_gap, gap_meas, ang_meas, bety, alfy, energy =  450e9):
    
    new_ang  = CRY_tw(new_gap, energy, beta_y = bety , alpha_y = alfy)[1]
    old_ang = CRY_tw(gap_meas, energy, beta_y = bety , alpha_y = alfy)[1]
    
    delta = new_ang - old_ang
    print("Delta ang: ", delta)
    
    return ang_meas - delta
    

In [ ]:
calculate_expected_chann_angle(new_gap = 4, gap_meas = TCCS_gap, ang_meas = TCCS_ch_angle_mesured, bety = TCCS_bety, alfy = TCCS_alfy) *1e6